In [87]:
import pandas as pd
from unidecode import unidecode
import numpy as np
import os

In [88]:
df = pd.DataFrame()
cols = ['DIST', 'MUN']
for chunk in pd.read_csv('data/raw/power/UCBT_PJ.csv', sep=';', engine='python', on_bad_lines='warn', encoding_errors='ignore', chunksize=100000):
    i = i + 1
    # chunk.to_parquet('data/processed/parquet/chunk'+str(i)+'.parquet')
    df = pd.concat([df, chunk[cols]], axis=0)


df = df.groupby('MUN').agg(pd.Series.mode).reset_index().rename(columns={'DIST': 'id_dist', 'MUN': 'id_city'})

In [89]:
cols = ['name', 'friendly_name', 'UF', 'id_dist']
df_dist = pd.read_excel('data/raw/power/AreaatuadistbaseBI.xlsx').rename(columns={'Código do Agente': 'id_dist', 'SIGLA': 'name'})[cols].set_index('id_dist')

In [90]:
df = pd.merge(df, df_dist, on='id_dist')


In [91]:
df_price = pd.read_excel('data/raw/power/data_tariffs.xlsx')
df_price = df_price.loc[(df_price['Subgrupo'] == 'A4') & 
                        (df_price['Modalidade'] == 'Verde') & 
                        (df_price['Base Tarifária'] == 'Tarifa de Aplicação') &
                        (df_price['Detalhe'] == 'Não se aplica') &
                        (df_price['Posto'] != 'Não se aplica')]

df_price['Início Vigência'] = pd.to_datetime(df_price['Início Vigência'])

# df_price.groupby(['Posto']).agg(list)
# df_price['TUSD', 'TE'].pivot_table()

/home/diego/Projects/UreaOpt/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [92]:
dist_list = df_price['Sigla'].unique()
new_df_price = pd.DataFrame(index=df_price['Sigla'].unique(), columns=['TUSD_Ponta', 'TUSD_Fora Ponta'])

for dist in dist_list:
    new_df_price.loc[dist, 'TUSD_Ponta'] = df_price['TUSD'].loc[(df_price['Sigla'] == dist) &
                                                    (df_price['Posto'] == 'Ponta')].max()
    new_df_price.loc[dist, 'TUSD_ForaPonta'] = df_price['TUSD'].loc[(df_price['Sigla'] == dist) &
                                                    (df_price['Posto'] == 'Fora ponta')].max()
    new_df_price.loc[dist, 'TE_Ponta'] = df_price['TE'].loc[(df_price['Sigla'] == dist) &
                                                (df_price['Posto'] == 'Fora ponta')].max()
    new_df_price.loc[dist, 'TE_ForaPonta'] = df_price['TE'].loc[(df_price['Sigla'] == dist) &
                                                (df_price['Posto'] == 'Fora ponta')].max()

new_df_price['TUSD'] = (new_df_price['TUSD_ForaPonta']*21 + new_df_price['TUSD_Ponta']*3) / 24
new_df_price['TE'] = (new_df_price['TE_ForaPonta']*21 + new_df_price['TE_Ponta']*3) / 24
new_df_price['grid_price'] = new_df_price['TE'] + new_df_price['TUSD']
new_df_price['friendly_name'] = new_df_price.index


In [93]:
# more fancy way of doing the above, but way too complicated with the several dates on prices.

# df = df_price.pivot(index='Sigla', columns='Posto', values=['TE', 'TUSD'])
# df['TE', 'Liq'] = (df['TE', 'Ponta']*3 + df['TE', 'Fora ponta']*21)/24
# df['TUSD', 'Liq'] = (df['TUSD', 'Ponta']*3 + df['TUSD', 'Fora ponta']*21)/24
# df['grid_price'] = df['TE', 'Liq'] + df['TUSD', 'Liq'] 

In [94]:
cols = ['grid_price', 'id_dist', 'UF', 'friendly_name']
df = pd.merge(df, new_df_price, on='friendly_name').set_index('id_city')

In [95]:
df = df[cols]


In [96]:
df.to_pickle('data/processed/power_grid_prices.p')